<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [ ]:
#Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" 
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# Import all necessary libraries.
import numpy as np
import sys
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import copy

# NomeroffNet path
# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.YoloV5Detector import Detector

detector = Detector()
detector.load()

from NomeroffNet.BBoxNpPoints import NpPointsCraft, getCvZoneRGB, convertCvZonesRGBtoBGR, reshapePoints
from NomeroffNet.MultiLineNPExtractor import CCraft
                                            

ccraft = CCraft()
npPointsCraft = NpPointsCraft()
npPointsCraft.load()

from NomeroffNet.MultiLineNPExtractor import convert_multiline_to_one_line, make_boxes

from NomeroffNet.OptionsDetector import OptionsDetector
from NomeroffNet.TextDetector import TextDetector


optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector.get_static_module("am")
textDetector.load("latest")

In [ ]:
import json
targetDir = '/home/datasets/multiline_2lines/am_2lines/train'
imgDir = os.path.join(targetDir,'img')
annDir = os.path.join(targetDir,'ann')

def add_np(fname, zone, region_id, count_lines):
    desc = fname.split('-')[0]
    height, width, c = zone.shape
    mpimg.imsave(os.path.join(imgDir, "{}.png".format(fname)), zone)
    data = {
      "description": desc,
      "name": fname,
      "region_id": region_id,
      "state_id": 2,
      "count_lines": count_lines,
      "size": {
        "width": width,
        "height": height
      },
      "moderation": {
        "isModerated": 0,
        "moderatedBy": "ApelSYN"
      }
    }
    with open(os.path.join(annDir, "{}.json".format(fname)), "w", encoding='utf8') as jsonWF:
            json.dump(data, jsonWF, ensure_ascii=False)

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
#rootDir = '/home/datasets/multiline_2lines/am_2lines/src/*'
#rootDir = '/home/datasets/multiline_2lines/2rowPlatesDataset/am_2row_2004/*'
rootDir = '/home/datasets/multiline_2lines/2rowPlatesDataset/am_2row_2014/*'



for img_path in glob.glob(rootDir):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    baseName = os.path.splitext(os.path.basename(img_path))[0]
    print(baseName)    
    print(img_path)
    
    
    targetBoxes = detector.detect_bbox(copy.deepcopy(img))
    targetBoxes = [[float(item) for item in target_box] for target_box in targetBoxes]
    
    
    print(img.shape)
    print(targetBoxes)
    img_to_show = copy.deepcopy(img)
    
    for targetBox in targetBoxes:
        cv2.rectangle(img_to_show, 
                      (int(targetBox[0]), int(targetBox[1])), 
                      (int(targetBox[2]), int(targetBox[3])), 
                      (0,120,255), 
                      3)

    
    all_points, all_mline_boxes = npPointsCraft.detect_mline(img, targetBoxes)
    all_mline_boxes = [[mline_box.tolist() for mline_box in mline_boxes] for mline_boxes in all_mline_boxes]
    
    # for  images/14.jpeg bug
    all_points = [ps for ps in all_points if len(ps)]
    
    make_boxes(img_to_show, all_points, (0, 255, 0))
#     plt.imshow(img_to_show)
#     plt.show()
    
#     print('all_points')
#     print(targetBoxes)
    # cut zones
    toShowZones = [getCvZoneRGB(img, reshapePoints(rect,1),0,0,2) for rect in all_points]
    zones = convertCvZonesRGBtoBGR(toShowZones)    

    # find standart
    regionIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    print(regionIds)
    print(regionNames)
    print(countLines)
    
    
    for idx, countLine in enumerate(countLines):
        print('idx:{}, countLine: {}'.format(idx, countLine) )
        if countLine > 1:
            zones[idx] = getCvZoneRGB(img, reshapePoints(all_points[idx],1),0,0,1.7)
            add_np(baseName,zones[idx],14,countLine)
            
    
#     zones = convert_multiline_to_one_line(img,
#                                           zones,
#                                           all_mline_boxes,
#                                           targetBoxes,
#                                           countLines,
#                                           regionNames)
    
#     for zone in zones:
#         plt.imshow(cv2.cvtColor(zone, cv2.COLOR_BGR2RGB))
#         plt.show()
                
#     # find text with postprocessing by standart  
#     textArr = textDetector.predict(zones, regionNames)
#     print(textArr)